In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [ ]:
!pip install jsonlines datasets git+https://github.com/google-research/bleurt.git evaluate sacrebleu

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-kbn9migb
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-kbn9migb
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456765 sha256=2eebf84068b035e59041b9043f96ac2a6a8763714a09249cd3f05ea2

In [ ]:
import tqdm
import jsonlines
import sys
import numpy as np

# Eval - BLEURT
from datasets import load_metric
bleurt = load_metric("bleurt", module_type="metric")
# Eval - chrf(++)
import evaluate
chrf = evaluate.load("chrf") # (word_order=2) for chrf++

<ipython-input-3-345e14c486b5>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleurt = load_metric("bleurt", module_type="metric")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bleurt contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bleurt/bleurt.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
t05e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t0.5-e0.01.jsonl'
t10e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t1-e0.01.jsonl'
t20e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t2.0-e0.01.jsonl'
t05e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t0.5-e0.02.jsonl'
t10e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t1-e0.02.jsonl'
t20e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t2.0-e0.02.jsonl'

t05e001 = []
t10e001 = []
t20e001 = []
t05e002 = []
t10e002 = []
t20e002 = []

# epsilon = 0.01
with jsonlines.open(t05e001_PATH) as f:
    for line in f.iter():
        t05e001.append(line)
with jsonlines.open(t10e001_PATH) as f:
    for line in f.iter():
        t10e001.append(line)
with jsonlines.open(t20e001_PATH) as f:
    for line in f.iter():
        t20e001.append(line)

# epsilon = 0.02
with jsonlines.open(t05e002_PATH) as f:
    for line in f.iter():
        t05e002.append(line)
with jsonlines.open(t10e002_PATH) as f:
    for line in f.iter():
        t10e002.append(line)
with jsonlines.open(t20e002_PATH) as f:
    for line in f.iter():
        t20e002.append(line)

In [ ]:
def BLEURT_mbrd(candidates):
  k_samples = len(candidates)
  score_matrix = np.zeros((k_samples, k_samples))

  for j1 in range(k_samples):
    references=[candidates[j1][0] for j2 in range(k_samples)]
    predictions=[candidates[j2][0] for j2 in range(k_samples)]
    scores = bleurt.compute(predictions=predictions, references=references)['scores']
    score_matrix[ : ,j1] = scores

  # remove votes for self
  for j1 in range(k_samples):
    for j2 in range(k_samples):
      if j1 == j2:
        score_matrix[j2,j1] = 0

  sum_scores = np.sum(score_matrix, axis=1)
  index = np.argmax(sum_scores)
  mbr_winner = candidates[index]

  return mbr_winner, index, sum_scores, score_matrix

# def CHARF_mbrd(candidates):
#   k_samples = len(candidates)
#   score_matrix = np.zeros((k_samples, k_samples))

#   for j1 in range(k_samples):
#     for j2 in range(k_samples):
#       references=[[candidates[j1][0]]]
#       predictions=[candidates[j2][0]]
#       scores = chrf.compute(predictions=predictions, references=references, word_order=2)['score']
#       score_matrix[j2,j1] = scores

#   # remove votes for self
#   for j1 in range(k_samples):
#     for j2 in range(k_samples):
#       if j1 == j2:
#         score_matrix[j2,j1] = 0

#   sum_scores = np.sum(score_matrix, axis=1)
#   index = np.argmax(sum_scores)
#   mbr_winner = candidates[index]

#   return mbr_winner, index, sum_scores, score_matrix


In [ ]:
num_candidates = 128
num_examples = 200

datasets = {'t05e001' : t05e001,
            't10e001' : t10e001,
            't20e001' : t20e001,
            't05e002' : t05e002,
            't10e002' : t10e002,
            't20e002' : t20e002}

In [ ]:
dataset_name = 't20e001'
dataset = datasets[dataset_name]
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    # dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-hyperparameter-selection/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)

100%|██████████| 200/200 [2:32:01<00:00, 45.61s/it]


In [ ]:
dataset_name = 't20e002'
dataset = datasets[dataset_name]
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    # dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-hyperparameter-selection/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)

100%|██████████| 200/200 [2:31:48<00:00, 45.54s/it]
